## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,62.4560,-114.3525,32.56,74,12,11.01,Few Clouds
1,1,Atuona,PF,-9.8000,-139.0333,77.97,82,100,14.09,Light Rain
2,2,Grand Gaube,MU,-20.0064,57.6608,79.38,74,9,3.00,Clear Sky
3,3,Petrovaradin,RS,45.2467,19.8794,69.94,35,100,13.51,Overcast Clouds
4,4,Hilo,US,19.7297,-155.0900,72.28,92,100,5.75,Light Rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Atuona,PF,-9.8000,-139.0333,77.97,82,100,14.09,Light Rain
2,2,Grand Gaube,MU,-20.0064,57.6608,79.38,74,9,3.00,Clear Sky
10,10,Arraial Do Cabo,BR,-22.9661,-42.0278,82.26,74,0,5.75,Clear Sky
14,14,Marsa Matruh,EG,31.3525,27.2453,81.88,36,47,12.10,Light Rain
15,15,Meulaboh,ID,4.1363,96.1285,78.17,89,100,9.33,Moderate Rain
16,16,Mahebourg,MU,-20.4081,57.7000,79.25,80,5,9.69,Clear Sky
20,20,Bairiki,KI,1.3292,172.9752,78.80,94,40,9.22,Light Intensity Shower Rain
23,23,Rikitea,PF,-23.1203,-134.9692,76.64,73,100,8.70,Overcast Clouds
26,26,Coahuayana,MX,18.7333,-103.6833,76.39,28,100,0.43,Overcast Clouds
28,28,Cape Town,ZA,-33.9258,18.4232,79.11,44,98,4.00,Overcast Clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                198
City                   198
Country                195
Lat                    198
Lng                    198
Max Temp               198
Humidity               198
Cloudiness             198
Wind Speed             198
Current Description    198
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                195
City                   195
Country                195
Lat                    195
Lng                    195
Max Temp               195
Humidity               195
Cloudiness             195
Wind Speed             195
Current Description    195
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Atuona,PF,77.97,Light Rain,-9.8000,-139.0333,
2,Grand Gaube,MU,79.38,Clear Sky,-20.0064,57.6608,
10,Arraial Do Cabo,BR,82.26,Clear Sky,-22.9661,-42.0278,
14,Marsa Matruh,EG,81.88,Light Rain,31.3525,27.2453,
15,Meulaboh,ID,78.17,Moderate Rain,4.1363,96.1285,
16,Mahebourg,MU,79.25,Clear Sky,-20.4081,57.7000,
20,Bairiki,KI,78.80,Light Intensity Shower Rain,1.3292,172.9752,
23,Rikitea,PF,76.64,Overcast Clouds,-23.1203,-134.9692,
26,Coahuayana,MX,76.39,Overcast Clouds,18.7333,-103.6833,
28,Cape Town,ZA,79.11,Overcast Clouds,-33.9258,18.4232,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get lat and lng
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"].title()
    except IndexError:
        print("Hotel not found. Skipping...")

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


In [9]:
# 7. Drop the rows where there is no Hotel Name.
# code from: https://bit.ly/37SbJhf
hotel_clean_df = hotel_df[hotel_df["Hotel Name"].str.strip().astype(bool)]
hotel_clean_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Atuona,PF,77.97,Light Rain,-9.8000,-139.0333,Villa Enata
2,Grand Gaube,MU,79.38,Clear Sky,-20.0064,57.6608,Veranda Paul Et Virginie Hotel & Spa
10,Arraial Do Cabo,BR,82.26,Clear Sky,-22.9661,-42.0278,Pousada Porto Praia
14,Marsa Matruh,EG,81.88,Light Rain,31.3525,27.2453,Beau Site
15,Meulaboh,ID,78.17,Moderate Rain,4.1363,96.1285,Kost Pria
16,Mahebourg,MU,79.25,Clear Sky,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
20,Bairiki,KI,78.80,Light Intensity Shower Rain,1.3292,172.9752,Tarawa Boutique Hotel
23,Rikitea,PF,76.64,Overcast Clouds,-23.1203,-134.9692,People Thankyou
26,Coahuayana,MX,76.39,Overcast Clouds,18.7333,-103.6833,El Rranchito Michuacan De Ocampos
28,Cape Town,ZA,79.11,Overcast Clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))